In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install langchain
!pip install unstructured
!pip install tiktoken

!pip install langchain-community
!pip install chromadb
!pip install sentence-transformers

!pip install scikit-learn
!pip install plotly

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminario RAG/data"

def load_documents():
  loader = DirectoryLoader(DATA_PATH, glob="**/*.md", recursive=True)
  documents = loader.load()
  return documents


documents = load_documents()
print(documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400, # 300 caratteri è la dimensione massima che può assumere un chunk
        chunk_overlap=100, # È il numero di caratteri sovrapposti tra un chunk e il successivo -> Serve per mantenere il contesto tra i blocchi, specialmente se il testo viene tagliato a metà di una frase o concetto.
        length_function=len,
        add_start_index=True,
    )
chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")


# source: da quale document è preso questo chunk; index: da dove inizia questo chunk
document = chunks[2]

print(document.page_content)
print(document.metadata)


In [ ]:
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import OpenAIEmbeddings
# import os
# import shutil

# from dotenv import load_dotenv
# load_dotenv()

# os.environ["OPENAI_API_KEY"] = "xxxxx"

# CHROMA_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminario RAG/chroma"

# if os.path.exists(CHROMA_PATH):
#   shutil.rmtree(CHROMA_PATH)

# # Creazione del DB vettoriale a partire dai chunks
# db = Chroma.from_documents(
#     chunks, embedding=OpenAIEmbeddings(), persist_directory=CHROMA_PATH
#     )
# db.persist()



In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import shutil


# A differenza di OpenAI, i modelli Hugging Face non richiedono chiavi API e non consumano crediti.
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # Piccolo, veloce e gratuito
)


#CHROMA_PATH = "/content/drive/MyDrive/Colab Notebooks/Seminario RAG/chroma"
CHROMA_PATH = "/content/chroma_temp"


if os.path.exists(CHROMA_PATH):
  shutil.rmtree(CHROMA_PATH)

# Creazione del DB vettoriale a partire dai chunks
db = Chroma.from_documents(
    chunks, embedding=embedding_model, persist_directory=CHROMA_PATH
)
db.persist()



In [ ]:
# Crea la funzione di embedding
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Ottiene il vettore embedding per la parola "apple"
vector = embedding_function.embed_query("apple")
print(vector)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


embed1 = embedding_function.embed_query("radio")
embed2 = embedding_function.embed_query("dog")

# Calcola la similarità tra i due embedding
similarity = cosine_similarity([embed1], [embed2])[0][0]
print(f"Cosine similarity: {similarity}")

In [ ]:
### VISUALIZZARE EMBEDDINGS IN UNO SPAZIO A 3 DIMENSIONI
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1. Frasi da visualizzare
texts = ["apple", "orange", "banana", "fruit", "laptop", "computer", "AI", "machine learning"]

# 2. Calcola gli embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = [embedding_model.embed_query(text) for text in texts]

# 3. Riduzione a 3 dimensioni con PCA
pca = PCA(n_components=3)
embeddings_3d = pca.fit_transform(embeddings)

# 4. Visualizzazione con matplotlib 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

for i, text in enumerate(texts):
    x, y, z = embeddings_3d[i]
    ax.scatter(x, y, z)
    ax.text(x, y, z, text)

ax.set_title("Visualizzazione 3D degli Embedding")
plt.show()


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# Frasi da visualizzare
texts = ["apple", "orange", "banana", "fruit", "laptop", "computer", "AI", "machine learning"]

# Calcola gli embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = [embedding_model.embed_query(text) for text in texts]

# Riduci a 3 dimensioni con PCA
pca = PCA(n_components=3)
embeddings_3d = pca.fit_transform(embeddings)

# Visualizzazione interattiva 3D con Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=embeddings_3d[:, 0],
    y=embeddings_3d[:, 1],
    z=embeddings_3d[:, 2],
    mode='markers+text',
    text=texts,
    textposition="top center",
    marker=dict(size=8, opacity=0.8),
)])

fig.update_layout(
    title='Embedding 3D Visualization',
    scene=dict(
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        zaxis_title='PCA 3'
    )
)

fig.show()


In [ ]:
# Dobbiamo trovare i nostri chunks nel db vettoriale che più probabilmente rispondono alla domanda che diamo in input

query_text = "Cosa dedica la rivista Life ad Andy Warhol?"

results = db.similarity_search_with_relevance_scores(query_text, k=3)

if not results or results[0][1] < 0.2: # soglia di accettazione
    print("Nessun risultato rilevante trovato.") # Il risultato può essere influito dalla dimensione dei chunk (magari hanno contesto troppo piccolo)
else:
  for doc, score in results:
      print("Score:", score)
      print("Document content:\n", doc.page_content)
      print("---")


In [ ]:
# Trovati i chunk rilevanti, possiamo dobbiamo creare una risposta rilevante a partire da questi chunk

from langchain.prompts import ChatPromptTemplate


query_text = "Cosa dedica la rivista Life ad Andy Warhol?"


# Crea un PROMPT TEMPLATE ( ti crei un bel prompt naive inserendoci il chunk (o i chunks) come contesto)
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {query}
"""
# Unisci i contenuti testuali dei documenti trovati
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

# Crea il template e formatta il prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, query=query_text)

print(prompt) # Prompt finale che andrà in pasto all'LLM

In [ ]:
# Dai la risposta finale con LLM

from transformers import pipeline

# Con GPT
# model = ChatOpenAI()
# response_text = model.predict(prompt)


from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
prompt = "Cosa dedica la rivista Life ad Andy Warhol?"
response = generator(prompt, max_length=100, do_sample=True)
print(response[0]['generated_text'])



In [ ]:
# Fornisci un riferimento alle fonti da cui ha tratto la risposta
response_text = response[0]['generated_text']

sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)
